In [1]:
import numpy as np
import pandas as pd
import re
import requests
from fractions import Fraction
from bs4 import BeautifulSoup

In [2]:
df = pd.read_csv('./nlp_data/IMDB_REVIEW_202104P5.csv', parse_dates=[2, 3])

In [3]:
hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

In [4]:
def para_handler(paragraph):
    regex = re.compile(r"<br/?>", re.IGNORECASE)
    newtext = re.sub(regex, '\n', str(paragraph))
    CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(CLEANR, '', newtext)

In [5]:
def scrape_review(url, name, hdr):
    r = requests.get(url, headers=hdr)
    soup = BeautifulSoup(r.text, 'html.parser')
    titles = soup.find_all('h3')
    text = ''
    for title in titles:
        if title.span != None:
            if title.text.replace(title.span.text, '')[:-1]==name:
                text = para_handler(title.find_next_sibling('p'))
    return text

In [6]:
def scrape_movie_rvw(movie_name, data, hdr):
    # req = requests.get(data, headers=data
    soup = BeautifulSoup(data, 'html.parser')
    rv_title = []
    usr = []
    usr_url = []
    rv_date = []
    rv = []
    rv_len = []
    exception = []
    is_spoilerwarn = []
#     rv_rating = []
#     pos_rat = []
#     neg_rat = []
    try:
        titles = soup.find_all('a', {'class': 'title'})
        for title in titles:
            rv_title.append(title.text[1:-1])
            try:
                isspoiler = title.find_next_sibling('span')
                is_spoilerwarn.append('Yes')
            except Exception as e:
                is_spoilerwarn.append('No')
                exception.append(e)
                print(movie_name)
                print(title)
                print(e)
                pass
    except Exception as e:
        rv_title.append(np.nan)
        exception.append(e)
        print(movie_name)
        print(e)
        pass
    try:
        users = soup.find_all('span', {'class': 'display-name-link'})
        for user in users:
            try:
                rv_date.append(user.find_next_sibling('span').text)
            except Exception as e:
                rv_date.append(np.nan)
                exception.append(e)
                print(user)
                print(movie_name)
                print(e)
                pass
            try:
                usr.append(user.text)
            except Exception as e:
                usr.append(np.nan)
                exception.append(e)
                print(user)
                print(movie_name)
                print(e)
                pass
            url = ''
            try:
                for name in user.find_all('a', href=True):
                    url = 'http://imdb.com/'+name['href']
                usr_url.append(url)
            except Exception as e:
                usr_url.append(np.nan)
                exception.append(e)
                print(user)
                print(movie_name)
                print(e)
                pass
    except Exception as e:
        usr.append(np.nan)
        exception.append(e)
        print(movie_name)
        print(e)
        pass

    try:
        reviews = soup.findAll('div', {'class': ['text show-more__control clickable', 'text show-more__control']})
    #     print(reviews)
        for review in reviews:
            rv.append(para_handler(review))
            rv_len.append(len(para_handler(review)))
    except Exception as e:
        rv.append(np.nan)
        rv_len.append(np.nan)
        exception.append(e)
        print(movie_name)
        print(e)
        pass
#     try:
#         spoilers = soup.find_all('span', {'class': 'spoiler-warning'})
#         for review in reviews:
#             is_spoilerwarn.append('Yes')
#     except Exception as e:
#         exception.append(e)
#         is_spoilerwarn.append('No')
#         pass
#     ratings = soup.findAll('div', {'class': 'ipl-ratings-bar'})
#     for rating in ratings:
#         rv_rating.append(float(Fraction(rating.span.text.split()[0]))*10)
#     helpfulness = soup.findAll('div', {'class': 'actions text-muted'})
#     for helpful in helpfulness:
#         pos_neg = [int(s) for s in helpful.text.split() if s.isdigit()]
#         pos_rat.append(pos_neg[0])
#         neg_rat.append(pos_neg[1]-pos_neg[0])
#     return rv_title, usr, usr_url, rv_date, rv, rv_len, pos_rat, neg_rat
    return rv_title, usr, usr_url, rv_date, rv, rv_len, is_spoilerwarn, exception

In [13]:
data = open('./data1.html', 'r')
soup = BeautifulSoup(data, 'html.parser')

In [14]:
titles = soup.find_all('a', {'class': 'title'})

In [19]:
isspoiler = titles[8].find_next_sibling('span')

In [20]:
isspoiler

<span class="spoiler-warning">Warning: Spoilers</span>

In [8]:
scrape_movie_rvw('White Boy Rick', './data3.html', hdr)

/opt/anaconda3/lib/python3.8/site-packages/bs4/__init__.py:346: MarkupResemblesLocatorWarning: "./data3.html" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


([], [], [], [], [], [], [], [])

In [7]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

# driver = webdriver.Chrome(ChromeDriverManager().install())
# wait = WebDriverWait(driver, 10)
# driver.get("http://imdb.com/title/tt0385887/reviews")
# i = 0
# while True:
#     try:
#         loadmoreBtn = driver.find_element("xpath", '//*[@id="load-more-trigger"]')
#         i += 1
#         time.sleep(2)
#         loadmoreBtn.click()
#         time.sleep(5)
#     except Exception as e:
#         print(e)
#         break
# print(i)

# file = open('data.html', 'w')
# file.write(driver.page_source)
# file.close()

# driver.close()

In [8]:
def load_all_page(url, output_file):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    wait = WebDriverWait(driver, 10)
    driver.get(url)
    while True:
        try:
            loadmoreBtn = driver.find_element("xpath", '//*[@id="load-more-trigger"]')
            time.sleep(2)
            loadmoreBtn.click()
            time.sleep(5)
        except Exception as e:
            print(e)
            break

    file = open(output_file, 'w')
    file.write(driver.page_source)
    file.close()

    driver.close()

In [9]:
def handle_data(output_file, df, hdr):
#     new_df = pd.DataFrame(columns=['user', 'user_url', 'review_date', 'review_title', 'review', 'review_len', 'pos_rating', 'neg_rating'])
    new_df = pd.DataFrame(columns=['user', 'user_url', 'review_date', 'review_title', 'review', 'review_len', 'is_spoilerwarn'])
    dup_len = []
    e = []
    for i in range(len(df)):
        load_all_page(df['imdb_url'][i], output_file)
        data = open(output_file, 'r')
#         rv_title, usr, usr_url, rv_date, rv, rv_len, pos_rat, neg_rat = scrape_movie_rvw(data, hdr)
        rv_title, usr, usr_url, rv_date, rv, rv_len, is_spoilerwarn, exception = scrape_movie_rvw(df['MOJO_title'][i], data, hdr)
        dup_len.append(len(usr))
#         temp_df = pd.DataFrame(np.array([usr, usr_url, rv_date, rv_title, rv, rv_len, pos_rat, neg_rat]).T, columns=['user', 'user_url', 'review_date', 'review_title', 'review', 'review_len', 'pos_rating', 'neg_rating'])
        temp_df = pd.DataFrame(np.array([usr, usr_url, rv_date, rv_title, rv, rv_len, is_spoilerwarn]).T, columns=['user', 'user_url', 'review_date', 'review_title', 'review', 'review_len', 'is_spoilerwarn'])
        new_df = pd.concat([new_df, temp_df], ignore_index=True)
        e += exception
    df = df.loc[df.index.repeat(dup_len)].reset_index(drop=True)
    df['user'] = new_df['user'].values
    df['user_url'] = new_df['user_url'].values
    df['review_date'] = new_df['review_date'].values
    df['review_title'] = new_df['review_title'].values
    df['review'] = new_df['review'].values
    df['review_len'] = new_df['review_len'].values
    df['is_spoilerwarn'] = new_df['is_spoilerwarn'].values
#     df['review_rating'] = new_df['review_rating'].values
#     df['pos_rating'] = new_df['pos_rating'].values
#     df['neg_rating'] = new_df['neg_rating'].values
    return df, exception

In [9]:
# def handle_file(output_file, df, hdr):
# #     new_df = pd.DataFrame(columns=['user', 'user_url', 'review_date', 'review_title', 'review', 'review_len', 'pos_rating', 'neg_rating'])
#     new_df = pd.DataFrame(columns=['user', 'user_url', 'review_date', 'review_title', 'review', 'is_spoilerwarn', 'review_len'])
#     dup_len = []
#     e = []
#     for i in range(len(df)):
#         # load_all_page(df['imdb_url'][i], output_file)
#         data = open(output_file, 'r')
# #         rv_title, usr, usr_url, rv_date, rv, rv_len, pos_rat, neg_rat = scrape_movie_rvw(data, hdr)
#         rv_title, usr, usr_url, rv_date, rv, rv_len, is_spoilerwarn, exception = scrape_movie_rvw(df['MOJO_title'][i], data, hdr)
#         dup_len.append(len(usr))
# #         temp_df = pd.DataFrame(np.array([usr, usr_url, rv_date, rv_title, rv, rv_len, pos_rat, neg_rat]).T, columns=['user', 'user_url', 'review_date', 'review_title', 'review', 'review_len', 'pos_rating', 'neg_rating'])
#         temp_df = pd.DataFrame(np.array([usr, usr_url, rv_date, rv_title, rv, rv_len, is_spoilerwarn]).T, columns=['user', 'user_url', 'review_date', 'review_title', 'review', 'review_len', 'is_spoilerwarn'])
#         new_df = pd.concat([new_df, temp_df], ignore_index=True)
#         e += exception
#     df = df.loc[df.index.repeat(dup_len)].reset_index(drop=True)
#     df['user'] = new_df['user'].values
#     df['user_url'] = new_df['user_url'].values
#     df['review_date'] = new_df['review_date'].values
#     df['review_title'] = new_df['review_title'].values
#     df['review'] = new_df['review'].values
#     df['review_len'] = new_df['review_len'].values
#     df['is_spoilerwarn'] = new_df['is_spoilerwarn'].values
# #     df['review_rating'] = new_df['review_rating'].values
# #     df['pos_rating'] = new_df['pos_rating'].values
# #     df['neg_rating'] = new_df['neg_rating'].values
#     return df, exception

In [13]:
data1 = pd.read_csv('./df1.csv', parse_dates=['FRI_RELEASE_DAYBO', 'TUE_V_RELEASE'])
df1, exception1 = handle_data('./data1.html', data1, hdr)

[WDM] - ====== WebDriver manager ======


2022-09-27 01:31:23,784 INFO ====== WebDriver manager ======


[WDM] - Current google-chrome version is 105.0.5195


2022-09-27 01:31:23,845 INFO Current google-chrome version is 105.0.5195


[WDM] - Get LATEST chromedriver version for 105.0.5195 google-chrome


...

2022-09-27 05:06:58,576 INFO Current google-chrome version is 105.0.5195


[WDM] - Get LATEST chromedriver version for 105.0.5195 google-chrome


2022-09-27 05:06:58,578 INFO Get LATEST chromedriver version for 105.0.5195 google-chrome


[WDM] - Driver [/Users/percyd/.wdm/drivers/chromedriver/mac64/105.0.5195.52/chromedriver] found in cache


2022-09-27 05:06:58,653 INFO Driver [/Users/percyd/.wdm/drivers/chromedriver/mac64/105.0.5195.52/chromedriver] found in cache
Message: element not interactable
  (Session info: chrome=105.0.5195.125)
Stacktrace:
0   chromedriver                        0x000000010269e788 chromedriver + 4515720
1   chromedriver                        0x00000001026229d3 chromedriver + 4008403
2   chromedriver                        0x00000001022b4feb chromedriver + 413675
3   chromedriver                        0x00000001022eda1e chromedriver + 645662
4   chromedriver                        0x00000001022e1861 chromedriver + 596065
5   chromedriver                        0x00000001023097d2 chromedriver + 759762
6   chromedriver                        0x00000001022e1075 chromedriver + 594037
7   chromedriver                        0x000000010230992e chromedriver + 760110
8   chromedriver                        0x000000010231cbd9 chromedriver + 838617
9   chromedriver                        0x000000010230960

In [14]:
data0 = pd.read_csv('./df0.csv', parse_dates=['FRI_RELEASE_DAYBO', 'TUE_V_RELEASE'])
df0, exception0 = handle_data('./data.html', data0, hdr)

[WDM] - ====== WebDriver manager ======


2022-09-27 05:08:34,448 INFO ====== WebDriver manager ======


[WDM] - Current google-chrome version is 105.0.5195


2022-09-27 05:08:34,492 INFO Current google-chrome version is 105.0.5195


[WDM] - Get LATEST chromedriver version for 105.0.5195 google-chrome


...

2022-09-27 06:39:13,977 INFO Get LATEST chromedriver version for 105.0.5195 google-chrome


[WDM] - Driver [/Users/percyd/.wdm/drivers/chromedriver/mac64/105.0.5195.52/chromedriver] found in cache


2022-09-27 06:39:14,037 INFO Driver [/Users/percyd/.wdm/drivers/chromedriver/mac64/105.0.5195.52/chromedriver] found in cache
Message: element not interactable
  (Session info: chrome=105.0.5195.125)
Stacktrace:
0   chromedriver                        0x00000001028ce788 chromedriver + 4515720
1   chromedriver                        0x00000001028529d3 chromedriver + 4008403
2   chromedriver                        0x00000001024e4feb chromedriver + 413675
3   chromedriver                        0x000000010251da1e chromedriver + 645662
4   chromedriver                        0x0000000102511861 chromedriver + 596065
5   chromedriver                        0x00000001025397d2 chromedriver + 759762
6   chromedriver                        0x0000000102511075 chromedriver + 594037
7   chromedriver                        0x000000010253992e chromedriver + 760110
8   chromedriver                        0x000000010254cbd9 chromedriver + 838617
9   chromedriver                        0x000000010253960

/var/folders/tl/jg9ncc9x4vnfwb17tjw2ms480000gn/T/ipykernel_47546/1803501279.py:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  temp_df = pd.DataFrame(np.array([usr, usr_url, rv_date, rv_title, rv, rv_len, is_spoilerwarn]).T, columns=['user', 'user_url', 'review_date', 'review_title', 'review', 'review_len', 'is_spoilerwarn'])


ValueError: Shape of passed values is (7, 1), indices imply (7, 7)

In [ ]:
data2 = pd.read_csv('./df2.csv', parse_dates=['FRI_RELEASE_DAYBO', 'TUE_V_RELEASE'])
df2, exception2 = handle_data('./data2.html', data2, hdr)

In [ ]:
data3 = pd.read_csv('./df3.csv', parse_dates=['FRI_RELEASE_DAYBO', 'TUE_V_RELEASE'])
df3, exception3 = handle_data('./data3.html', data3, hdr)

In [ ]:
data4 = pd.read_csv('./df3.csv', parse_dates=['FRI_RELEASE_DAYBO', 'TUE_V_RELEASE'])
df4, exception4 = handle_data('./data4.html', data4, hdr)

In [ ]:
data5 = pd.read_csv('./df5.csv', parse_dates=['FRI_RELEASE_DAYBO', 'TUE_V_RELEASE'])
df5, exception5 = handle_data('./data5.html', data5, hdr)

In [ ]:
data6 = pd.read_csv('./df6.csv', parse_dates=['FRI_RELEASE_DAYBO', 'TUE_V_RELEASE'])
df6, exception6 = handle_data('./data6.html', data6, hdr)

In [ ]:
data7 = pd.read_csv('./df7.csv', parse_dates=['FRI_RELEASE_DAYBO', 'TUE_V_RELEASE'])
df7, exception7 = handle_data('./data7.html', data7, hdr)